In [ ]:
import pandas as pd
import numpy as np
import h5py
import zipfile

import torch
from dataclasses import make_dataclass

from importlib import reload
from src.datasets import UKBSnpLevelDatasetH5OneHot

from src.util import load_config
from src.graphs import GeneOntology
from src.vnn_trainer import GenoVNNLightning, FastVNNLightning, FastVNNLitReg

import matplotlib.pyplot as plt
import seaborn as sns

from src import xai
from importlib import reload

from tqdm import tqdm

sns.set_context("talk")
torch.cuda.is_available()

# Loading config and model

In [ ]:
gene_bim_file = "/cluster/project/beltrao/gankin/vnn/data/ukb_gene.bim"
ontology_file = "/cluster/project/beltrao/gankin/vnn/snp_vnn/data/NEST_UKB_snp_onto.txt"

model_ckpt = "/cluster/project/beltrao/gankin/vnn/snp_vnn/data/checkpoints/gout/epoch=2-step=5859.ckpt"

In [ ]:
config_file = "src/config/gout_config.yaml"
config = load_config(config_file)
config["feature_dim"] = 3

args = make_dataclass(
    "DataclassFromConfig", [(k, type(v)) for k, v in config.items()]
)(**config)

gene_bim_df = pd.read_csv(gene_bim_file, sep="\t")

snp_id_map = {
    snp: ind
    for snp, ind in zip(
        gene_bim_df["snp"].unique(), range(0, len(gene_bim_df["snp"].unique()))
    )
}

graph = GeneOntology(
    snp_id_map,
    ontology_file,
    child_node="snp",
)

In [ ]:
go_vnn_model = FastVNNLightning.load_from_checkpoint(model_ckpt, args = args, graph=graph)

# Compute gene level attributions?